# EconML

## Python
### Enquête Emploi en Continu 2015

1. [Experimental Setup](python/eec15/01_experimental_setup.ipynb)
2. [Linear Regressions](python/eec15/02_linear_regressions.ipynb)
3. [Logit Regressions](python/eec15/03_logit_regressions.ipynb)
4. [Probit Regressions](python/eec15/04_probit_regressions.ipynb)
5. [Simple Tree](python/eec15/05_simple_tree.ipynb)
6. [Random Forest](python/eec15/06_random_forest.ipynb)


*Use this **[template notebook](python/eec15/template.ipynb)** for additional experiments.*

## R
- [Notebook](r/notebook.ipynb)